## New York AirBNB Exploratory Analysis 

### Importing Packages and preparing data

In [ ]:
## Data source: https://insideairbnb.com/get-the-data/

In [ ]:
import os
#Checking for current directory and switching to project folder
current_directory = os.getcwd()
print(current_directory)
os.chdir('/Users/smarr/Documents/Data Sets/AirBNBAnalysis')

In [ ]:
# Verifying Directory was switched
os.getcwd()

In [ ]:
import pandas as pd

In [ ]:
## Importing listings data and cleaning the price column to ensure successful numerical analysis
listings = pd.read_csv('../AirBNBAnalysis/data/listings_clean.csv')
listings['price']= listings['price'].replace('[\$,]', '', regex=True).astype(float)
listings.to_csv('listings_clean.csv', index=False)

In [ ]:
# Testing
listings.head()

In [ ]:
# Importing SQL python package to work with the SQL database, establishing connection between python and SQL
from sqlalchemy import create_engine
engine = create_engine('postgresql://smarr@localhost:5432/airbnb_analysis', echo=False)
conn = engine.connect()

In [ ]:
#Submitting our listings dataframe to SQL
listings.to_sql('listings', engine, if_exists='replace', index=False)

In [ ]:
## We are going to partition the ensuing calendar dataset since it is extremely large and jupyter cannot feasibly process it all

chunksize = 500000
sample_size = 50000
samples = []

for chunk in pd.read_csv('../AirBNBAnalysis/data/calendar.csv', chunksize=chunksize):
    # Random sample 2% from each chunk (adjust if needed)
    sample = chunk.sample(frac=0.02, random_state=42)
    samples.append(sample)
    
    # Stop once we hit the target
    if sum(len(s) for s in samples) >= sample_size:
        break

df_sampled = pd.concat(samples).head(sample_size)

In [ ]:
## Saving it to a smaller csv
df_sampled.to_csv('../AirBNBAnalysis/data/calendar_sampled.csv', index=False)

In [ ]:
df_sampled['price'] = df_sampled['price'].replace(r'[\$,]', '', regex=True).astype(float)

In [ ]:
df_sampled.to_sql('calendar', engine, if_exists='replace', index=False)

In [ ]:
# Testing data frame for adjusted price column
df_sampled.head()

In [ ]:
## call engine.dispose() or ensure .close() is used on connections to prevent table lock in postgresql

### Pivot table for understanding price change over time across neighbourhoods

In [ ]:
df = pd.read_sql_query("""
    SELECT 
        TO_CHAR(c.date::DATE, 'YYYY-MM') AS month,
        l.neighbourhood_cleansed,
        ROUND(AVG(c.price)::numeric, 2) AS avg_price
    FROM calendar c
    JOIN listings l ON c.listing_id = l.id
    WHERE c.available = 't'
    GROUP BY TO_CHAR(c.date::DATE, 'YYYY-MM'), l.neighbourhood_cleansed
""", conn)


In [ ]:
pivot_df = df.pivot(index='neighbourhood_cleansed', columns='month', values='avg_price')
pivot_df.head()

In [ ]:
pivot_df_cleaned = pivot_df.dropna()
pivot_df_cleaned
## Some color would look great on this

In [ ]:
styled_df=pivot_df_cleaned.style.background_gradient(
    cmap='RdYlGn_r',  # Red for high, green for low
    axis=1            # Apply column-wise
)
styled_df

### Adding reviews csv file for sentiment analysis (Python and PostgreSQL)

In [ ]:
dfReviews = pd.read_csv("../AirBNBAnalysis/data/reviews.csv", low_memory=False)

In [ ]:
dfReviews.to_sql('reviews', engine, if_exists='replace', index=False)

In [ ]:
dfReviews.head()

In [ ]:
## Drop Null values as they will make it more difficult to do sentiment analysis
dfReviews=dfReviews.dropna(subset=['comments'])

In [ ]:
## Checking length of Dataframe for verification purposes
len(dfReviews)

In [ ]:
## Make all comments lower-case for standardizing purposes
dfReviews['comments']=dfReviews['comments'].str.lower()

### Sentiment Scoring

In [ ]:
from textblob import TextBlob

In [ ]:
## Define Function
def get_sentiment(text):
    return TextBlob(text).sentiment.polarity

In [ ]:
## Apply it to Airbnb comments
dfReviews['Sentiment']=dfReviews['comments'].apply(get_sentiment)

In [ ]:
dfReviews[['listing_id','Sentiment']].head()
## Doesn't quite get us our average sentiment by listing

### Average sentiment and count by listing ID

In [ ]:
sentimentAvgandCount_by_listing=dfReviews.groupby('listing_id').agg(avg_sentiment=('Sentiment','mean'),sentiment_count=('Sentiment','count')).reset_index()
sentimentAvgandCount_by_listing

In [ ]:
## Merge with Listings data
CombinedDF=listings.merge(sentimentAvgandCount_by_listing,left_on='id',right_on='listing_id')

In [ ]:
CombinedDF

In [ ]:
sentimentAvgandCount_by_listing.sort_values(by='avg_sentiment', ascending=False).head(10)
## Want to get listings that have a sentiment count greater than 10, to get a substantive analysis

In [ ]:
sentimentAvgandCount_by_listing = sentimentAvgandCount_by_listing[sentimentAvgandCount_by_listing['sentiment_count']>10]

In [ ]:
sentimentAvgandCount_by_listing.sort_values(by='avg_sentiment', ascending=False).head(10)
## Listing ID is pretty abstract. I want to see the URl and the neighbourhood that each listing is in.

In [ ]:
sentimentAvgandCount_by_listingNew=sentimentAvgandCount_by_listing.merge(listings[['neighbourhood_cleansed', 'listing_url','id']], left_on = 'listing_id', right_on='id')

In [ ]:
## Now lets run the group by and get a better idea of the area and general quality of the listing
sentimentAvgandCount_by_listingNew.sort_values(by='avg_sentiment', ascending=False).head(10).drop(columns=['id'], errors='ignore')

In [ ]:
## Taking a look at the above listing url's will quickly demonstrate why they each score farily high on the average sentimeent column.

### Average sentiment and count by neighbourhood (Additional Merge is needed)

In [ ]:
CombinedDF_NeighbourhoodGroupby=dfReviews.merge(listings[['id','neighbourhood_cleansed']], left_on = 'listing_id', right_on='id')

In [ ]:
CombinedDF_NeighbourhoodGroupby

In [ ]:
## Drop id_x and y values
CombinedDF_NeighbourhoodGroupby = CombinedDF_NeighbourhoodGroupby.drop(columns=['id_x', 'id_y'], errors='ignore')

In [ ]:
CombinedDF_NeighbourhoodGroupby

In [ ]:
sentimentAvgandCount_by_neighbourhood=CombinedDF_NeighbourhoodGroupby.groupby('neighbourhood_cleansed').agg(avg_sentiment=('Sentiment','mean'),sentiment_count=('Sentiment','count')).reset_index()

In [ ]:
sentimentAvgandCount_by_neighbourhood.sort_values(by='avg_sentiment', ascending=False).head(10)

In [ ]:
# For a more reliable analysis, lets include only the neighbourhood scontaining more than 50 sentiment_count entries
sentimentAvgandCount_by_neighbourhood = sentimentAvgandCount_by_neighbourhood[sentimentAvgandCount_by_neighbourhood['sentiment_count']>50]

In [ ]:
sentimentAvgandCount_by_neighbourhood.sort_values(by='avg_sentiment', ascending=False).head(10)
## Not surprisignly, we see neighbourhoods like Vinegar Hill, Financial District, and Rockaway beach score relatively highly. 
## I'm sure Tottenville is a great place too - I've yet to see Staten Island